In [1]:
#step 1: install/upgrade the latest genai SDK
%pip install google-genai --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 11.4 MB/s eta 0:00:00


In [2]:
#import the genai library
from google import genai

In [3]:
#step2: AIStudio: read the api key from the user data
from google.colab import userdata
client = genai.Client(api_key=userdata.get("GEMINI_KEY"))

#If you want to read from environment keys
#import os
#client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [4]:
#step2: Vertex AI: read the api key from the user data

#for vertex, you need to have a service account or login/auth
#login as the current user for colab testing purposes
from google.colab import auth
auth.authenticate_user()

#then configure the client with the project and location settings

#Uncomment the following if you are running on Vertex setup

#using the colab userdata
#from google.colab import userdata
#client = genai.Client(vertexai=True, #ensure to turn on VertexAI
#                      project=userdata.get("GOOGLE_CLOUD_PROJECT"),
#                      location=userdata.get("GOOGLE_CLOUD_REGION")) # e.g., 'us-central1'

#or use the folliwing if you want to read from environment keys
#import os
#client = genai.Client(vertexai=True,
#                      project=os.environ["GOOGLE_CLOUD_PROJECT"],
#                      location=os.environ["GOOGLE_CLOUD_REGION"]) # e.g., 'us-central1'

In [5]:
from google import genai
from google.protobuf.struct_pb2 import Struct, Value

# Define the function's description and parameters using OpenAPI schema
get_current_weather = genai.types.FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location",
    parameters=Struct(
        fields={
            "type": Value(string_value="object"),  # Indicates parameters are an object
            "properties": Value(  # <--- THIS IS THE KEY CHANGE! Wrap the Struct in a Value
                struct_value=Struct(
                    fields={
                        "location": Value(  # 'location' itself is also a Value wrapping a Struct
                            struct_value=Struct(
                                fields={
                                    "type": Value(string_value="string"),
                                    "description": Value(string_value="The city and state, e.g. San Francisco, CA")
                                }
                            )
                        )
                    }
                )
            ),
            "required": Value(
                list_value=Value.ListValue(
                    values=[
                        Value(string_value="location")
                    ]
                )
            )
        }
    )
)

# Group declarations into a Tool
weather_tool = genai.types.Tool(function_declarations=[get_current_weather])

print(weather_tool)

AttributeError: ListValue

In [6]:
#step 6: Try a custom generation configuration

#ensure types import
from google.genai import types

my_generation_config = types.GenerateContentConfig(
    temperature=0.9,
    top_p=0.9,
    top_k=40,
    max_output_tokens=1500,
    system_instruction = "You are a helpful assistant that talks like a Pirate. Be concise."
)

# Generate content using the custom configuration
model_name = "models/gemini-2.5-flash-preview-05-20"
prompt='What is a good name for a flower shop that specializes in selling bouquets of dried flowers?'

print(f"Generating content with custom config (Temperature: {my_generation_config.temperature}, Max Tokens: {my_generation_config.max_output_tokens}):\n")
response = client.models.generate_content(
    model=model_name,
    contents=prompt,
    config=my_generation_config
)
print(response.text)

Generating content with custom config (Temperature: 0.9, Max Tokens: 1500):

Ahoy! Try "The Everlastin' Bloom."
